In [1]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
#import everythin for a sequential keras model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation

In [2]:
#load dataset
train_features_data=pd.read_csv('train_features.csv')
labels_data=pd.read_csv('train_labels.csv')
#extract features with pandas groupby method
train_data=pd.DataFrame(train_features_data.groupby(['pid']).mean())
train=train_data.reset_index().drop(columns=['pid', 'Time']).fillna(train_data.mean())
labels=labels_data.drop(columns=['pid'])
labels = labels[['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2']]
#split into train and test set
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.2, random_state=42)

In [60]:
train_features_data=pd.read_csv('train_features.csv')
labels_data=pd.read_csv('train_labels.csv')
labels_data=labels_data.drop(columns=['pid'])

In [ ]:
train_features_data=pd.read_csv('train_features.csv')
labels_data=pd.read_csv('train_labels.csv')
labels_copied = pd.DataFrame()
for i in range(len(labels_data)):
    for j in range(12):
        labels_copied = labels_copied.append(labels_data.loc[i,:])
train_features_data_new = pd.concat([train_features_data, labels_copied])

train = train_features_data_new.reset_index().drop(columns=['pid', 'Time']).fillna(method='ffill')

In [57]:
labels = train[['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2']]
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.2, random_state=42)

4

In [17]:
#make a sequential model
model = Sequential([
    Dense(units=128, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(units=128, activation='relu'),
    Dense(units=10, activation='sigmoid'),
])
#compile the model

model.summary()


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 128)               4608      
                                                                 
 dense_25 (Dense)            (None, 128)               16512     
                                                                 
 dense_26 (Dense)            (None, 10)                1290      
                                                                 
Total params: 22,410
Trainable params: 22,410
Non-trainable params: 0
_________________________________________________________________


In [ ]:

model.compile(optimizer='sgd', loss='binary_focal_crossentropy', metrics=[tf.keras.metrics.AUC()])
model.fit(X_train, y_train, epochs=40, batch_size=32)

In [7]:
y_pred = model.predict(X_test)
err=np.sqrt(mean_squared_error(y_pred,y_test))
print(err)

0.9103684579000128


In [39]:
display(y_train[y_train == 1].sum()/y_train.shape[0]*100)
display(y_train.shape)


LABEL_BaseExcess          26.671492
LABEL_Fibrinogen           7.297973
LABEL_AST                 23.933930
LABEL_Alkalinephos        23.631219
LABEL_Bilirubin_total     24.058963
LABEL_Lactate             19.840748
LABEL_TroponinI           10.107923
LABEL_SaO2                23.256120
LABEL_Bilirubin_direct     3.382469
LABEL_EtCO2                6.528034
dtype: float64

(15196, 10)

Predict Separately

In [15]:
model = Sequential([
    Dense(units=128, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(units=128, activation='relu'),
    Dense(units=1, activation='sigmoid'),
])
model.compile(optimizer='sgd', loss='binary_focal_crossentropy', metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
#load dataset
train_features_data=pd.read_csv('train_features.csv')
labels_data=pd.read_csv('train_labels.csv')
#extract features with pandas groupby method
train_data=pd.DataFrame(train_features_data.groupby(['pid']).mean())
train=train_data.reset_index().drop(columns=['pid', 'Time']).fillna(train_data.mean())
labels=labels_data.drop(columns=['pid'])
labels = labels[['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2']]
#split into train and test set
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(train, labels.iloc[:,i], test_size=0.2, random_state=42)
    model.fit(X_train, y_train, epochs=10, batch_size=32)
    y_pred = model.predict(X_test)
    err=np.sqrt(mean_squared_error(y_pred,y_test.iloc[:,i]))
    print(err)

In [4]:
#load dataset
train_features_data=pd.read_csv('train_features.csv')
labels_data=pd.read_csv('train_labels.csv')
#extract features with pandas groupby method
train_data=pd.DataFrame(train_features_data.groupby(['pid']).mean())
train=train_data.reset_index().drop(columns=['pid', 'Time']).fillna(train_data.mean())
labels=labels_data.drop(columns=['pid'])
labels = labels[['LABEL_Bilirubin_total']]
#split into train and test set

X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.2, random_state=42)
#SVC versuch
from sklearn.svm import SVC
svc = SVC(kernel='sigmoid',probability=True, class_weight='balanced')
svc.fit(X_train, np.ravel(y_train))
y_pred = svc.predict(X_test)


[0. 0. 1. ... 1. 1. 1.]


In [10]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test['LABEL_Bilirubin_total'], y_pred))

0.47775730455383


## Subtask 2

In [32]:
#load dataset
train_features_data=pd.read_csv('train_features.csv')
labels_data=pd.read_csv('train_labels.csv')
#extract features with pandas groupby method
train_data=pd.DataFrame(train_features_data.groupby(['pid']).mean())
train=train_data.reset_index().drop(columns=['pid', 'Time']).fillna(train_data.mean())
labels=labels_data.drop(columns=['pid'])
labels = labels['LABEL_Sepsis']
#split into train and test set
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.2, random_state=42)

In [5]:
train_features_data=pd.read_csv('train_features.csv')
labels_data=pd.read_csv('train_labels.csv')
# add labels with matching pid to features
labels_copied = pd.DataFrame()
labels_copied = labels_data.loc[labels_data.index.repeat(12)]
labels_copied = labels_copied.drop(columns=['pid'])

In [6]:
#join labels on the right of the features
train_features_data_new = train_features_data.join(labels_copied, how='right')
train_features_data_new = train_features_data_new.drop(columns=['pid', 'Time']).reset_index(drop=True)

In [18]:
X = train_features_data.drop(columns=['pid', 'Time']).reset_index(drop=True).fillna(train_features_data.mean())
y = labels_copied['LABEL_Sepsis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
model = Sequential([
    Dense(units=64, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=1, activation='sigmoid'),
])
model.compile(optimizer='sgd', loss='binary_focal_crossentropy', metrics=[tf.keras.metrics.AUC()])

model.fit(X_train, np.ravel(y_train), epochs=20, batch_size=2048)

y_pred = model.predict(X_test)

Epoch 1/20
90/90 [==============================] - 1s 3ms/step - loss: 3.1921 - auc_13: 0.4954
Epoch 2/20
90/90 [==============================] - 0s 3ms/step - loss: 0.0880 - auc_13: 0.4952
Epoch 3/20
90/90 [==============================] - 0s 3ms/step - loss: 0.0732 - auc_13: 0.4994
Epoch 4/20
90/90 [==============================] - 0s 4ms/step - loss: 0.0692 - auc_13: 0.4978
Epoch 5/20
90/90 [==============================] - 0s 5ms/step - loss: 0.0686 - auc_13: 0.4965
Epoch 6/20
90/90 [==============================] - 1s 7ms/step - loss: 0.0665 - auc_13: 0.4980
Epoch 7/20
90/90 [==============================] - 1s 6ms/step - loss: 0.0663 - auc_13: 0.4997
Epoch 8/20
90/90 [==============================] - 1s 6ms/step - loss: 0.0661 - auc_13: 0.4987
Epoch 9/20
90/90 [==============================] - 1s 6ms/step - loss: 0.0652 - auc_13: 0.4986
Epoch 10/20
90/90 [==============================] - 0s 6ms/step - loss: 0.0796 - auc_13: 0.4982
Epoch 11/20
90/90 [====================